### Connect to your workspace


#### create conda environment from yaml file

In [ ]:
!conda env create -f myenv.yml

### Initialize Workspace
Initialize a workspace object from json configuration file.

In [1]:
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.36.0


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

azure_ml loaded


### Create and register the dataset

In [3]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

azureml_globaldatasets - Default = False
workspaceworkingdirectory - Default = False
workspaceblobstore - Default = True
workspaceartifactstore - Default = False
workspacefilestore - Default = False


In [ ]:
default_ds.upload_files(files=['./data/diamonds.csv'],
                       target_path='diamond-data/', 
                       overwrite=True,
                       show_progress=True)

In [4]:
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diamond-data/*.csv'))

# Display the first 5 rows as a Pandas dataframe
tab_data_set.take(5).to_pandas_dataframe()

,Column1,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [ ]:
# Register the tabular dataset
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                        name='diamond dataset',
                                        description='diamond data',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)
    
print('Datasets registered')

### Create Environnement

In [5]:
from azureml.core import Environment

tf_env = Environment.from_existing_conda_environment(name="tf_env", conda_environment_name="az_env")
tf_env.save_to_directory(path="./tf_env", overwrite=True)

Exporting conda specifications for existing conda environment: az_env


In [ ]:
tf_env.register(workspace=ws)

In [7]:
from azureml.core import Experiment

experiment_name = 'dimond_experiment'
experiment = Experiment(workspace=ws, name=experiment_name)

### Create and run experiment

In [8]:
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails

# Get the training dataset
diamond_ds = ws.datasets.get("diamond dataset")

src = ScriptRunConfig(source_directory='./training',
                      script='model.py',
                      compute_target='local',
                      arguments = ['--input-data', diamond_ds.as_named_input('training_data')],
                      environment=tf_env)

run = experiment.submit(config=src)

# Show the running experiment run in the notebook widget
RunDetails(run).show()
# Block until the experiment run has completed
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'dimond_experiment_1639396756_a9e45179',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-12-13T12:05:21.061357Z',
 'endTimeUtc': '2021-12-13T12:08:46.277592Z',
 'services': {},
 'warnings': [{'message': 'Specifying a version of the certifi python package is not recommended. It receives regular updates with new versions of root certificates and old versions will not be compatible with new python installations.'}],
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'afa937ca-63a3-46c2-83ec-cbb6934d65d3',
  'azureml.git.repository_uri': 'https://github.com/amine-akrout/diamond-price-prediction.git',
  'mlflow.source.git.repoURL': 'https://github.com/amine-akrout/diamond-price-prediction.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'e8a632cb9fe5b8164a4a9bb64efe284b2ee98641',
  'mlflow.source.git.commit': 'e8a632cb9fe5b8164a4a9bb64efe284b2ee98641',
  'azureml.git.dirty': 'False'},


In [9]:
# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

RMSE 1586.727294921875
MAPE 15.646038055419922


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/16172_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
outputs/model/keras_metadata.pb
outputs/model/saved_model.pb
outputs/model/variables/variables.data-00000-of-00001
outputs/model/variables/variables.index


### Register the saved model

In [10]:
from azureml.core.model import Model
from azureml.core.resource_configuration import ResourceConfiguration


model = run.register_model(model_path='./outputs/model/', 
                        model_name='diamond_model',
                        tags={'Training context':'Script'},
                        properties={'RMSE': run.get_metrics()['RMSE'], 'MAPE': run.get_metrics()['MAPE']})

print("Model name: ", model.name)
print("Model id: ", model.id)
print("Model version: ", model.version)

Model name:  diamond_model
Model id:  diamond_model:6
Model version:  6


In [11]:
   from azureml.core.model import Model
   model_path = Model.get_model_path('diamond_model', _workspace = ws)
   print(model_path)

'azureml-models\\diamond_model\\6\\model'

### Test model locally

In [12]:
import tensorflow as tf
reloaded_model = tf.keras.models.load_model(model_path)

# Perform inference
raw_data = {
    'carat': 0.23,
    'cut': 'Ideal',
    'color': 'E',
    'clarity': 'SI2',
    'depth': 61.5,
    'table': 55.0,
    'x': 3.95,
    'y': 3.98,
    'z': 2.43,
}
volume = raw_data['x']*raw_data['y']*raw_data['z']
raw_data['volume'] =  volume

input_dict = {name: tf.convert_to_tensor([value]) for name, value in raw_data.items()}
predictions = reloaded_model.predict(input_dict)
predicted = predictions[0]

print("estimated price :", predicted[0], "+/- 9%")

estimated price : 431.65622 +/- 9%


### Deploy the model

#### create deployment folder

In [ ]:
import os

project_folder = './deployment'
os.makedirs(project_folder, exist_ok=True)

#### create scoring script 

In [173]:
%%writefile score.py

import json
import os
import tensorflow as tf
import numpy as np

from azureml.core.model import Model
import logging
logging.basicConfig(level=logging.DEBUG)


def init():
    global model
    model_path = Model.get_model_path('diamond_model')
    model = tf.keras.models.load_model(model_path)


def run(data):
    test = json.loads(data)
    print(f"received data {test}")
    input_dict = {name: tf.convert_to_tensor([value]) for name, value in test.items()}
    predictions = model.predict(input_dict)
    predicted = predictions[0]
    result = str(round(predicted[0],2))
    return result


Overwriting score.py


### Configure inference environment

Create and register an inference environment from yaml file

In [139]:
inf_env = Environment.from_conda_specification(name = "inf_env",
                                             file_path = "./inf_env.yml")

In [ ]:
inf_env.register(workspace=ws)

In [176]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(
    entry_script='./score.py', 
    environment=inf_env
)

### Create ACI configuration
Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container.

In [177]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=1, 
    tags={"data": "diamond",  "method" : "tensorflow"}, 
    description='Predict diamond price with tensorflow'
)

### Deploy in ACI

In [178]:
%%time
aci_service_name = 'diamond-price-aci-svc'
aci_service = Model.deploy(workspace=ws,
                           name=aci_service_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aci_config,
                           overwrite=True)
aci_service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-13 17:10:13+01:00 Creating Container Registry if not exists.
2021-12-13 17:10:13+01:00 Registering the environment.
2021-12-13 17:10:15+01:00 Use the existing image.
2021-12-13 17:10:15+01:00 Generating deployment configuration.
2021-12-13 17:10:16+01:00 Submitting deployment to compute.
2021-12-13 17:10:19+01:00 Checking the status of deployment diamond-price-aci-svc..
2021-12-13 17:12:00+01:00 Checking the status of inference endpoint diamond-price-aci-svc.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Wall time: 2min 39s


Get the scoring web service's logs

In [ ]:
print(aci_service.get_logs())

Get the scoring web service's HTTP endpoint, which accepts REST client calls.

In [81]:
print(aci_service.scoring_uri)

http://affdaeb4-9abd-473d-a1c2-e35c67dd4b0f.francecentral.azurecontainer.io/score


In [99]:
print(aci_service.swagger_uri)

http://affdaeb4-9abd-473d-a1c2-e35c67dd4b0f.francecentral.azurecontainer.io/swagger.json


### Test deployed service
Now, we can test the deployed model with a sample data

In [180]:
import requests
import json

# send a random row from the test set to score

data = {
    'carat': 0.23,
    'cut': 'Ideal',
    'color': 'E',
    'clarity': 'SI2',
    'depth': 61.5,
    'table': 55.0,
    'x': 3.95,
    'y': 3.98,
    'z': 2.43,
}
volume = data['x']*data['y']*data['z']
data['volume'] =  volume
# input_data = bytes(raw_data, encoding='utf8')
data = json.dumps(data)

headers = {'Content-Type':'application/json'}

response  = requests.post(aci_service.scoring_uri, data, headers=headers)

print("POST to url", aci_service.scoring_uri)
print("input data:", data)
print("response ", response.json())
print("response ", response.text)

POST to url http://affdaeb4-9abd-473d-a1c2-e35c67dd4b0f.francecentral.azurecontainer.io/score
input data: {"carat": 0.23, "cut": "Ideal", "color": "E", "clarity": "SI2", "depth": 61.5, "table": 55.0, "x": 3.95, "y": 3.98, "z": 2.43, "volume": 38.20203}
response  431.66
response  "431.66"
